# A scratch pad to run model inference manually


In [1]:

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
import transformers


from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


/media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [2]:
# load my code
%load_ext autoreload
%autoreload 2


from src.extraction.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset
from src.models.load import load_model
from src.datasets.intervene import create_cache_interventions 


In [3]:
# load config, model, dataset, invtervention
N_fit_examples=10
batch_size=2
ds_name='amazon_polarity'
cfg = ExtractConfig(max_examples=(20, 20), model='TheBloke/Mistral-7B-Instruct-v0.1-GPTQ', prompt_format='llama2')

model, tokenizer = load_model(cfg.model)
model

honesty_rep_reader = create_cache_interventions(model, tokenizer, cfg)

N=sum(cfg.max_examples)
ds_tokens = load_preproc_dataset(ds_name, tokenizer, N=N, seed=cfg.seed, num_shots=cfg.num_shots, max_length=cfg.max_length, prompt_format=cfg.prompt_format)


2023-10-27 17:14:08.461 | INFO     | src.models.load:verbose_change_param:19 - changing pad_token_id from None to 0
2023-10-27T17:14:08.461621+0800 INFO changing pad_token_id from None to 0
2023-10-27 17:14:08.462 | INFO     | src.models.load:verbose_change_param:19 - changing truncation_side from right to left
2023-10-27T17:14:08.462733+0800 INFO changing truncation_side from right to left


## Generate

In [ ]:
r = ds_tokens.with_format('torch')[0]

# r['input_ids']
r.keys()


In [ ]:
s = model.generate(r['input_ids'][None, :], attention_mask=r['attention_mask'][None, :])
tokenizer.decode(s[0])
